# Instalar com.microsoft.sqlserver:mssql-jdbc:10.2.0.jre8 no databricks 

%pip install azure-identity==1.19.0
%pip install azure-storage-file-datalake==12.18.0


In [ ]:
# Define the connection string for SQL Server
jdbc_url = "jdbc:sqlserver://sql-new-tapir.database.windows.net:1433;database=SampleDB"
connection_properties = {
  "user" : "azureadmin",
  "password" : "Satc@1234",
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

query = "(SELECT table_name FROM INFORMATION_SCHEMA.TABLES) AS query"

# Read data from SQL Server
df = spark.read.jdbc(url=jdbc_url, table=query, properties=connection_properties)
df.toPandas()

In [ ]:
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core.exceptions import ResourceExistsError
from urllib.parse import quote_plus

# ADLS_ACCOUNT_NAME
account_name = "datalakec4056b1df75aa671"

# ADLS_FILE_SYSTEM_NAME
file_system_name = "landing-zone"

# ADLS_DIRECTORY_NAME
directory_name = "a"

# ADLS_SAS_TOKEN
sas_token = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-11-27T21:52:16Z&st=2024-11-27T13:52:16Z&spr=https&sig=7l8kb5ju4S24tgoyk%2F0Y8bLsu2zCeXHryL9rc4k1YM8%3D"

file_system_client = DataLakeServiceClient(
    account_url=f"https://{account_name}.dfs.core.windows.net", 
    credential=sas_token,
    api_version="2020-02-10"
)

try:
    directory_client = file_system_client.get_file_system_client(file_system_name).get_directory_client(directory_name)
    directory_client.create_directory()
except ResourceExistsError:
    print(f"O diretório '{directory_name}' já existe.")

# Tudo certo com o de baixo 
Ele adiciona as coisas na camadas landing-zone na pasta "a"
Se faltar libs instalar com 
%pip install azure-identity==1.19.0
%pip install azure-storage-file-datalake==12.18.0

In [ ]:
import pandas as pd
from azure.identity import DefaultAzureCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core.exceptions import ResourceExistsError
from urllib.parse import quote_plus

# ADLS_ACCOUNT_NAME
account_name = "datalakec4056b1df75aa671"

# ADLS_FILE_SYSTEM_NAME
file_system_name = "landing-zone"

# ADLS_DIRECTORY_NAME
directory_name = "a"

# ADLS_SAS_TOKEN
sas_token = ""

jdbc_url = "jdbc:sqlserver://sql-new-tapir.database.windows.net:1433;database=SampleDB"
connection_properties = {
  "user" : "azureadmin",
  "password" : "Satc@1234",
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
schema = "dbo"

query = f"(SELECT table_name FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{schema}') AS query"
df = spark.read.jdbc(url=jdbc_url, table=query, properties=connection_properties).toPandas()

file_system_client = DataLakeServiceClient(
    account_url=f"https://{account_name}.dfs.core.windows.net", 
    credential=sas_token,
    api_version="2020-02-10"
)

try:
    directory_client = file_system_client.get_file_system_client(file_system_name).get_directory_client(directory_name)
    directory_client.create_directory()
except ResourceExistsError:
    print(f"O diretório '{directory_name}' já existe.")

# Para cada tabela encontrada, ler os dados e carregar para o Azure Data Lake Storage
for index, row in df.iterrows():
    table_name = row["table_name"]
    query2 = f"(SELECT * FROM {schema}.{table_name}) as query"
    df_table = spark.read.jdbc(url=jdbc_url, table=query2, properties=connection_properties).toPandas()
    
    file_client = directory_client.get_file_client(f"{table_name}.csv")
    data = df_table.to_csv(index=False).encode()
    file_client.upload_data(data, overwrite=True)
    print(f"Dados da tabela '{table_name}' carregados com sucesso.")